In [3]:
import sys
import os
path_to_AOC_helpers = "/Users/connor/Desktop/Coding Problems/Advent_of_code/2024"
# Add the parent directory to the sys.path
sys.path.append(os.path.abspath(path_to_AOC_helpers))

from collections import defaultdict, deque, Counter, namedtuple
from functools import reduce, cache, cmp_to_key
from heapq import heappush, heappop, heapify
from itertools import permutations, combinations, product
from math import gcd, sqrt, factorial, ceil, floor
from sortedcontainers import SortedDict, SortedList, SortedSet
#import PyRival
from typing import TypeAlias
from math import prod
import re
import numpy as np
import sys
import pyomo.environ as pymo
import pyomo.opt as pyopt

from AOC_helpers import *
sys.setrecursionlimit(10000)

input_file = 'input.txt'
test_file = 'test.txt'

with open(input_file, 'r') as file:
    lines = file.readlines()
    blocks = get_blocks(lines)
    
reg_vals = [scan_nums(line)[0] for line in blocks[0]]



program = scan_nums(blocks[-1][0])
print(f'Our input is ')
print(reg_vals)
print(program)
print()
literal = lambda x: x # litteral operand

def combo_op(x, reg_vals):
    if x < 0:
        print('error')
    if x < 4:
        return x 
    elif x == 4:
        return reg_vals[0]
    elif x == 5:
        return reg_vals[1]
    elif x == 6: 
        return reg_vals[2]
    else:
        print(f'error: Op 7 encountered!')
        return - 1
    
def opcode(code, operand, reg_vals, result, i, jump):
    if code == 0:
        num = reg_vals[0]
        denom = 2 ** combo_op(operand, reg_vals)
        res = num // denom
        reg_vals[0] = res 
    elif code == 1:
        reg_vals[1] = reg_vals[1] ^ literal(operand)
    elif code == 2:
        reg_vals[1] = combo_op(operand, reg_vals) % 8
    elif code == 3:
        if reg_vals[0] != 0:
            i = literal(operand)
            jump = True
    elif code == 4:
        reg_vals[1]  = reg_vals[1] ^ reg_vals[2] 
    elif code == 5:
        result.append(combo_op(operand, reg_vals) % 8 )
    elif code == 6: 
        num = reg_vals[0] 
        denom = 2 ** combo_op(operand, reg_vals)
        res = num // denom
        reg_vals[1] = res
    elif code == 7:
        num = reg_vals[0] 
        denom = 2 ** combo_op(operand, reg_vals)
        res = num // denom
        reg_vals[2] = res
    return i, jump 

def run_program(program, reg_vals):
    instruction_pointer = 0
    jump = False
    result = []
    n = len(program)
    while instruction_pointer < n:
       
        code = program[instruction_pointer]
        operand = program[instruction_pointer +  1]
        
        instruction_pointer, jump = opcode(code, operand, reg_vals, result, instruction_pointer, jump)
        
        if jump:
            jump = False 
            continue
        else: 
            instruction_pointer += 2 
    
    print(f'resulting registers {reg_vals} and output {result}')
    return ','.join([str(c) for c in result])
            

run_program(program, reg_vals)
            
            
    
    
# So im tasked with simulating this device 
# fill registers initially 
print(len(program))

Our input is 
[65804993, 0, 0]
[2, 4, 1, 1, 7, 5, 1, 4, 0, 3, 4, 5, 5, 5, 3, 0]

resulting registers [0, 6, 0] and output [5, 1, 4, 0, 5, 1, 0, 2, 6]
16


In [18]:
"""
# Two types of operand
#              [0-3]: values 0,.., 3) let the
               4 : rep reg A 
               5. rep reg B 
               6. reps reg C
               7 does not appear
Instructions
    0 -> division: numerator in A, denom -> 2^ combo operand 
    1 -> bitwise or of B and litteral op, stores in B
    2 -> value of combo op mod 8 puts in B
    3 -> if A = 0, do nothing else, set pointer to value of litteral op, ins pointer does not jump 2
    4 -> bitwise or of B and C, store in B (ignores operand)
    5 -> ouputs value of combo op mod 8
    6 - > does 0 but stores in B
    7 - > does 0 but stores in C
    
    
 
  EX  2 6 -> bst with num in reg C store in B
  EX 
"""     

'\n# Two types of operand\n#              [0-3]: values 0,.., 3) let the\n               4 : rep reg A \n               5. rep reg B \n               6. reps reg C\n               7 does not appear\nInstructions\n    0 -> division: numerator in A, denom -> 2^ combo operand \n    1 -> bitwise or of B and litteral op, stores in B\n    2 -> value of combo op mod 8 puts in B\n    3 -> if A = 0, do nothing else, set pointer to value of litteral op, ins pointer does not jump 2\n    4 -> bitwise or of B and C, store in B (ignores operand)\n    5 -> ouputs value of combo op mod 8\n    6 - > does 0 but stores in B\n    7 - > does 0 but stores in C\n    \n    \n \n  EX  2 6 -> bst with num in reg C store in B\n  EX \n'

In [83]:
test1 = [0, 0, 9]
program1 = [2, 6]

test2 = [10, 0, 9]
program2 = [5,0,5,1,5,4]

test3 = [2024, 0, 0]
program3 = [0,1,5,4,3,0]

test4 = [2024, 29, 0]
program4 = [1,7]

test5 = [2024, 2024, 43690]
program5 = [4,0]

test6 = [117440, 0, 0]
program6 = [0,3,5,4,3,0]

run_program(program6, test6)
p

resulting registers [0, 0, 0] and output [0, 3, 5, 4, 3, 0]


'0,3,5,4,3,0'

In [ ]:
# ok now to find a value of A that gives the program back. 
# note that
"""
So our specific input is [2, 4, 1, 1, 7, 5, 1, 4, 0, 3, 4, 5, 5, 5, 3, 0]

Note the outputs occur at end. 
We output what's in B mod 8 each time. 
We end when A is empty. The only thing that modifies A is opcode 0.

We need at A to be such that we can divide it 16 ish times.
(2, 4) -> (B = A % 8)
(1, 1) -> (B = B ^ 1)
(7, 5) -> (C = A // 2 ** B)
(1, 4) -> (B = A^B)
(0, 3) -> A = A // 8
(4, 5) -> B = B ^ C
(5, 5) -> Ouput (B % 8)
(3, 0) -> If A is still there, repeat. 

# note, A is divided by 8 each time. S0 A in 8^15 < A < 8^16.

# What is in B? 
B = (((A % 8)^1^A)^(A // (2 ** (A % 8)^1)) 



"""
def output(A, program):
    
    